<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping for Indeed.com and Predicting Salaries

### Business Case Overview

You're working as a data scientist for a contracting firm that's rapidly expanding. Now that they have their most valuable employee (you!), they need to leverage data to win more contracts. Your firm offers technology and scientific solutions and wants to be competitive in the hiring market. Your principal wants you to

   - determine the industry factors that are most important in predicting the salary amounts for these data.

To limit the scope, your principal has suggested that you *focus on data-related job postings*, e.g. data scientist, data analyst, research scientist, business intelligence, and any others you might think of. You may also want to decrease the scope by *limiting your search to a single region.*

Hint: Aggregators like [Indeed.com](https://www.indeed.com) regularly pool job postings from a variety of markets and industries.

**Goal:** Scrape your own data from a job aggregation tool like Indeed.com in order to collect the data to best answer this question.

---

### Directions

In this project you will be leveraging a variety of skills. The first will be to use the web-scraping and/or API techniques you've learned to collect data on data jobs from Indeed.com or another aggregator. Once you have collected and cleaned the data, you will use it to address the question above.

### Factors that impact salary

To predict salary the most appropriate approach would be a regression model.
Here instead we just want to estimate which factors (like location, job title, job level, industry sector) lead to high or low salary and work with a classification model. To do so, split the salary into two groups of high and low salary, for example by choosing the median salary as a threshold (in principle you could choose any single or multiple splitting points).

Use all the skills you have learned so far to build a predictive model.
Whatever you decide to use, the most important thing is to justify your choices and interpret your results. *Communication of your process is key.* Note that most listings **DO NOT** come with salary information. You'll need to be able to extrapolate or predict the expected salaries for these listings.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10").

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters:

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [7]:
URL = "http://www.indeed.com/jobs?"

In [35]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook

pd.set_option('display.max_colwidth',1000)

In [43]:
## YOUR CODE HERE

title = []
company = []
rating = []
location = []
summary = []
age = []

for x in tqdm_notebook(range(10, 200, 10)):
    URL = "http://www.indeed.com/jobs?"
    PARAMS = dict(q='data scientist $20,000', l='New York', start={})

    r = requests.get(url=URL, params=PARAMS)
    soup = BeautifulSoup(r.text, 'html.parser')

    for posting in soup.find_all('div', attrs={'class':'result'}):
        try:
            title.append(posting.find('a', attrs={'class':'jobtitle'}).text.split())
        except:
            title.append(np.nan)
        try:
            company.append(posting.find('span', attrs={'class':'company'}).text.split())
        except:
            company.append(np.nan)
        try:
            rating.append(posting.find('span', attrs={'class':'ratingsDisplay'}).text.split())
        except:
            rating.append(np.nan)
        try:
            location.append(posting.find('span', attrs={'class':'location'}).text.split())
        except:
            location.append(np.nan)
        try:
            summary.append(posting.find('div', attrs={'class':'summary'}).text.split())
        except:
            summary.append(np.nan)
        try:
            age.append(posting.find('span', attrs={'class':'date'}).text.split())
        except:
            age.append(np.nan)

jobs = pd.DataFrame(dict(title=title,company=company,rating=rating,location=location,summary=summary))
jobs.head()

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is in a `span` with `class='salaryText'`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element='jobTitle'`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 
- Decide which other components could be relevant, for example the region or the summary of the job advert.

### Write 4 functions to extract each item: location, company, job, and salary.

Example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it.
    - Remember to use `try/except` if you anticipate errors.
- **Test** the functions on the results above and simple examples.

In [40]:
## YOUR CODE HERE

def extract_title_from_result(result):
    try:
        return result.find('a', attrs={'class':'jobtitle', 'data-tn-element':'jobTitle'}).text.split()
    except:
        np.nan
    
def extract_company_from_result(result):
    try:
        return result.find('span', attrs={'class':'company'}).text.split()
    except:
        return np.nan
        
def extract_rating_from_result(result):
    try:
        return result.find('span', attrs={'class':'ratingsDisplay'}).text.split()
    except:
        return np.nan
        
def extract_location_from_result(result):
    try:
        return result.find('span', attrs={'class':'location'}).text.split()
    except:
        return np.nan

def extract_summary_from_result(result):
    try:
        return result.find('div', attrs={'class':'summary'}).text.split()
    except:
        return np.nan

def extract_date_from_result(result):
    try:
        return result.find('span', attrs={'class':'date'}).text.split()
    except:
        return np.nan

def extract_salary_from_result(result):
    try:
        return result.find('span', attrs={'class':'salaryText'}).text.split()
    except:
        return np.nan


In [42]:
title = []
company = []
rating = []
location = []
summary = []
date = []
salary = []

for x in tqdm_notebook(range(10, 200, 10)):
    URL = "http://www.indeed.com/jobs?"
    PARAMS = dict(q='data scientist $20,000', l='New York', start={})

    r = requests.get(url=URL, params=PARAMS)
    soup = BeautifulSoup(r.text, 'html.parser')

    for posting in soup.find_all('div', attrs={'class':'result'}):
        title.append(extract_title_from_result(posting))
        company.append(extract_company_from_result(posting))
        rating.append(extract_rating_from_result(posting))
        location.append(extract_location_from_result(posting))
        summary.append(extract_summary_from_result(posting))
        date.append(extract_date_from_result(posting))
        salary.append(extract_salary_from_result(posting))

jobs = pd.DataFrame(dict(title=title,company=company,rating=rating,location=location,summary=summary,date=date,salary=salary))

print(jobs.shape)
jobs.head()

,title,company,rating,location,summary,date,salary
0,"[Data, Scientist,, Marketplace]",[Spotify],[4.3],"[New, York,, NY, 10011, (Flatiron, District, area)]","[Marketplace, is, the, home, for, Spotify’s, music, industry, products,, such, as, Spotify, for, Artists,, Spotify, Label, Analytics, and, Spotify, Publisher, Analytics.SPSSSoftware, package, created, by, IBM, used, for, statistical, analysis.SQLProgramming, language, used, for, interacting, with, data, held, in, a, relational, database.Data, ScienceKnowledge, or, experience, employing, various, data, science, methods,, algorithms,, and, programming, languages.RA, software, environment, and, language, for, statistical, computing.ScalaA, functional, programming, language, that, runs, on, the, JVM, (Java, Virtual, Machine).PythonAn, interpreted,, open-source, programming, language.+8, moreMathematics, DegreePostsecondary, degree, in, mathematics,, applied, mathematics,, probability,, or, number, theory., Statistics, DegreePostsecondary, degree, in, statistics., AnalyticsExperience, or, skill, with, finding, and, analyzing, patterns, in, data.Computer, ...]","[1, day, ago]",NaN
1,"[Data, Scientist, -, Trello]",[Atlassian],[4.5],"[New, York,, NY]","[Our, data, team, is, lean,, nimble,, and, remote-friendly.Partner, with, engineering,, product,, and, leadership, teams, focused, on, strategic, improvements, to, Trello.Disability, FriendlyIndividuals, with, disabilities, are, encouraged, to, apply.ConfluenceKnowledge, management, and, workplace, collaboration, software, developed, by, Atlassian.JiraIssue, tracking, and, project, management, software, by, Atlassian.RedshiftA, data, warehouse, product, that, is, part, of, the, larger, cloud, computing, platform,, Amazon, Web, Services.TableauData, visualization, software, built, for, business, intelligence, applications., Developed, by, Tableau.RA, software, environment, and, language, for, statistical, computing.+4, morePythonAn, interpreted,, open-source, programming, language.SQLProgramming, language, used, for, interacting, with, data, held, in, a, relational, database.RemoteA, job, that, can, be, performed, entirely, remotely., May, involve, on-site, ...]","[6, days, ago]",NaN
2,"[Associate, Data, Scientist]","[Direct-to-Consumer, and, International]",NaN,"[New, York,, NY]","[Supporting, the, Disney, Advertising, Sales, organization, within, our, new, Direct, To, Consumer, &, International, (DTCI), segment,, the, Data, Science, &, Analytics, team, are…OracleDatabase, used, for, online, transaction, processing, and, data, warehousing, developed, by, Oracle, Corporation.SQLProgramming, language, used, for, interacting, with, data, held, in, a, relational, database.HadoopA, framework, that, allows, for, the, distributed, processing, of, large, data, sets, across, clusters, of, computers, using, simple, programming, models.Data, MiningSkill, in, data, mining, techniques, used, to, identify, patterns, in, large, amounts, of, data.HiveData, warehouse, software.StatisticsKnowledge, or, experience, in, the, field, of, statistics,, from, basic, to, advanced.+5, moreCommunication, SkillsExperience, or, skill, effectively, exchanging, and, understanding, ...]","[1, day, ago]",NaN
3,"[Data, Scientist]","[Source, Enterprises]",[5.0],"[New, York,, NY]","[Work, with, our, industry, experts, to, gain, unrivaled, knowledge, at, both, the, strategic, level, (generating, bold, and, innovative, ideas, for, data, science-driven, growth), and…Writing, SkillsSkills,, knowledge,, and, experience, with, creating, written, content, through, any, medium.Flexible, ScheduleWork, schedule, that, allows, employees, to, vary, their, arrival, and, departure, times.MATLABA, programming, language, and, environment, used, for, numerical, and, technical, computation.CassandraOpen-source, noSQL, database, built, with, NoSQL, for, scalability, and, high, availability, and, developed, by, Apache.MongoDBOpen-source, NOSQL, document-oriented, database.RemoteA, job, t

Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search.
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different.

In [5]:
YOUR_CITY = ''

In [6]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 100 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', YOUR_CITY]):
    for start in range(0, max_results_per_city, 10):
        # Grab the results from the request (as above)
        # Append to the full set of results
        pass

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [7]:
## YOUR CODE HERE

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now.
1. Some of the entries may be duplicated.
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries.

In [8]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary.

In [9]:
## YOUR CODE HERE

### Save your results as a CSV

In [10]:
## YOUR CODE HERE

### Load in the the data of scraped salaries

In [11]:
## YOUR CODE HERE

### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median).

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choose the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries.

In [12]:
## YOUR CODE HERE

#### Thought experiment: What is the baseline accuracy for this model?

In [13]:
## YOUR CODE HERE

### Create a classification model to predict High/Low salary. 


- Start by ONLY using the location as a feature.
- Use at least two different classifiers you find suitable.
- Remember that scaling your features might be necessary.
- Display the coefficients/feature importances and write a short summary of what they mean.
- Create a few new variables in your dataframe to represent interesting features of a job title (e.g. whether 'Senior' or 'Manager' is in the title).
- Incorporate other text features from the title or summary that you believe will predict the salary.
- Then build new classification models including also those features. Do they add any value?
- Tune your models by testing parameter ranges, regularization strengths, etc. Discuss how that affects your models.
- Discuss model coefficients or feature importances as applicable.

In [14]:
## YOUR CODE HERE

### Model evaluation:

Your boss would rather tell a client incorrectly that they would get a lower salary job than tell a client incorrectly that they would get a high salary job. Adjust one of your models to ease his mind, and explain what it is doing and any tradeoffs.


- Use cross-validation to evaluate your models.
- Evaluate the accuracy, AUC, precision and recall of the models.
- Plot the ROC and precision-recall curves for at least one of your models.

In [15]:
## YOUR CODE HERE

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### Bonus:

- Answer the salary discussion by using your model to explain the tradeoffs between detecting high vs low salary positions. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.
- Obtain the ROC/precision-recall curves for the different models you studied (at least the tuned model of each category) and compare.

In [16]:
## YOUR CODE HERE

### Summarize your results in an executive summary written for a non-technical audience.
   
- Writeups should be at least 500-1000 words, defining any technical terms, explaining your approach, as well as any risks and limitations.

In [17]:
## YOUR TEXT HERE IN MARKDOWN FORMAT 

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### BONUS

Convert your executive summary into a public blog post of at least 500 words, in which you document your approach in a tutorial for other aspiring data scientists. Link to this in your notebook.

In [18]:
## YOUR LINK HERE IN MARKDOWN FORMAT 